Import some libraries required to execute

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for modelling the problem

Sepecially, we use XGboost model to train the given data


In [ ]:
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error, accuracy_score

In [ ]:
## import the given data 

Ascendum=pd.read_csv(r"/content/Ascendeum_Dataset2.csv")

Ascendum_final_data=Ascendum

In [ ]:
Ascendum.head()    ## checking the data 

,date,site_id,ad_type_id,geo_id,device_category_id,advertiser_id,order_id,line_item_type_id,os_id,integration_type_id,monetization_channel_id,ad_unit_id,total_impressions,total_revenue,viewable_impressions,measurable_impressions,revenue_share_percent
0,2019-06-30 00:00:00,351,10,187,2,84,3473,19,60,1,4,5174,16,0.0,2,16,1
1,2019-06-30 00:00:00,351,10,187,2,84,3473,19,58,1,4,5174,6,0.0,0,6,1
2,2019-06-30 00:00:00,351,10,147,2,84,3473,19,60,1,4,5174,4,0.0,0,4,1
3,2019-06-30 00:00:00,351,10,147,2,84,3473,19,58,1,4,5174,4,0.0,0,4,1
4,2019-06-30 00:00:00,348,10,147,3,79,3463,20,59,1,19,5138,4,0.0,0,4,1


In [ ]:
Ascendum.describe()    ## describing the data check mean std_dev etc...

,site_id,ad_type_id,geo_id,device_category_id,advertiser_id,order_id,line_item_type_id,os_id,integration_type_id,monetization_channel_id,ad_unit_id,total_impressions,total_revenue,viewable_impressions,measurable_impressions,revenue_share_percent
count,567291.000000,567291.000000,567291.000000,567291.000000,567291.000000,567291.000000,567291.000000,567291.000000,567291.0,567291.000000,567291.000000,567291.000000,567291.000000,567291.000000,567291.000000,567291.0
mean,347.107775,10.220578,110.226133,1.899669,262.084295,1951.273747,12.632081,57.217194,1.0,14.433716,5136.436645,33.673674,0.069740,12.838251,32.152818,1.0
std,2.930371,1.222863,66.177820,0.791982,627.325490,1652.545702,6.812571,1.956012,0.0,7.135874,62.508575,220.865448,0.713603,101.989184,207.096504,0.0
min,342.000000,10.000000,1.000000,1.000000,8.000000,45.000000,3.000000,15.000000,1.0,1.000000,5050.000000,0.000000,-0.154200,0.000000,0.000000,1.0
25%,345.000000,10.000000,39.000000,1.000000,79.000000,45.000000,4.000000,56.000000,1.0,4.000000,5099.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,346.000000,10.000000,122.000000,2.000000,88.000000,3438.000000,11.000000,57.000000,1.0,19.000000,5127.000000,2.000000,0.000000,0.000000,2.000000,1.0
75%,350.000000,10.000000,183.000000,3.000000,139.000000,3460.000000,19.000000,58.000000,1.0,19.000000,5168.000000,6.000000,0.005400,2.000000,6.000000,1.0
max,351.000000,17.000000,308.000000,5.000000,2646.000000,3489.000000,20.000000,60.000000,1.0,21.000000,5443.000000,14452.000000,83.822000,7392.000000,13756.000000,1.0


In [ ]:
Ascendum.shape

(567291, 17)

In [ ]:
Ascendum.info()   ## get the info of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567291 entries, 0 to 567290
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   date                     567291 non-null  datetime64[ns]
 1   site_id                  567291 non-null  int64         
 2   ad_type_id               567291 non-null  int64         
 3   geo_id                   567291 non-null  int64         
 4   device_category_id       567291 non-null  int64         
 5   advertiser_id            567291 non-null  int64         
 6   order_id                 567291 non-null  int64         
 7   line_item_type_id        567291 non-null  int64         
 8   os_id                    567291 non-null  int64         
 9   integration_type_id      567291 non-null  int64         
 10  monetization_channel_id  567291 non-null  int64         
 11  ad_unit_id               567291 non-null  int64         
 12  total_impression

In [ ]:
## Handling the date column

Ascendum['date']=pd.to_datetime(Ascendum['date'],errors='coerce')

In [ ]:
Ascendum.head()

,date,site_id,ad_type_id,geo_id,device_category_id,advertiser_id,order_id,line_item_type_id,os_id,integration_type_id,monetization_channel_id,ad_unit_id,total_impressions,total_revenue,viewable_impressions,measurable_impressions,revenue_share_percent
0,2019-06-30,351,10,187,2,84,3473,19,60,1,4,5174,16,0.0,2,16,1
1,2019-06-30,351,10,187,2,84,3473,19,58,1,4,5174,6,0.0,0,6,1
2,2019-06-30,351,10,147,2,84,3473,19,60,1,4,5174,4,0.0,0,4,1
3,2019-06-30,351,10,147,2,84,3473,19,58,1,4,5174,4,0.0,0,4,1
4,2019-06-30,348,10,147,3,79,3463,20,59,1,19,5138,4,0.0,0,4,1


In [ ]:
Ascendum.isnull().sum()   ## checking the null value

date                       0
site_id                    0
ad_type_id                 0
geo_id                     0
device_category_id         0
advertiser_id              0
order_id                   0
line_item_type_id          0
os_id                      0
integration_type_id        0
monetization_channel_id    0
ad_unit_id                 0
total_impressions          0
total_revenue              0
viewable_impressions       0
measurable_impressions     0
revenue_share_percent      0
dtype: int64

In [ ]:
Ascendum=Ascendum.drop_duplicates()   ## droppinng the duplicate values

In [ ]:
Ascendum.shape    

(544127, 17)

In [ ]:
## Adding the columns like weekday and fraction column

Ascendum['weekday']=Ascendum['date'].apply(lambda x:x.weekday())
Ascendum['view_percentage_fraction']=np.where(Ascendum['measurable_impressions']!=0, Ascendum['viewable_impressions']/Ascendum['measurable_impressions'],0)

In [ ]:
Ascendum.nunique()    ## count the unique values

date                           30
site_id                        10
ad_type_id                      2
geo_id                        219
device_category_id              5
advertiser_id                  23
order_id                       75
line_item_type_id               7
os_id                           7
integration_type_id             1
monetization_channel_id         5
ad_unit_id                    132
total_impressions            1979
total_revenue               10099
viewable_impressions         1159
measurable_impressions       1884
revenue_share_percent           1
weekday                         7
view_percentage_fraction    12673
dtype: int64

In [ ]:
## Since integration_type_id and revenue_share_percent has unique value so, it doesnt help much further

Acsendum_corr=Ascendum
Ascendum = Ascendum.drop(['integration_type_id','revenue_share_percent'],axis=1)
Ascendum=Ascendum.drop(['order_id','line_item_type_id'],axis=1)     ## order_id and line_item_id is not important to

KeyError: ignored

In [ ]:
Ascendum.head()

,date,site_id,ad_type_id,geo_id,device_category_id,advertiser_id,os_id,integration_type_id,monetization_channel_id,ad_unit_id,total_impressions,total_revenue,viewable_impressions,measurable_impressions,revenue_share_percent,weekday,view_percentage_fraction
0,2019-06-30,351,10,187,2,84,60,1,4,5174,16,0.0,2,16,1,6,0.125
1,2019-06-30,351,10,187,2,84,58,1,4,5174,6,0.0,0,6,1,6,0.000
2,2019-06-30,351,10,147,2,84,60,1,4,5174,4,0.0,0,4,1,6,0.000
3,2019-06-30,351,10,147,2,84,58,1,4,5174,4,0.0,0,4,1,6,0.000
4,2019-06-30,348,10,147,3,79,59,1,19,5138,4,0.0,0,4,1,6,0.000


In [ ]:
Ascendum.shape

(544127, 15)

Modelling the data with outliers and check the metrics


In [ ]:
Ascendum['CPM'] = np.where(Ascendum['measurable_impressions']!=0, (Ascendum['total_revenue']*100/Ascendum['measurable_impressions'])*1000,0)

In [ ]:
X=Ascendum.drop(columns=['date','CPM'])         ## Assign the data in variables
Y=Ascendum['CPM']

In [ ]:
X_train=X.loc[Ascendum.date<pd.Timestamp('06-22-2019')]                     ## partition the data based on the date in train and test split
X_val=X.loc[Ascendum.date>=pd.Timestamp('06-22-2019')]
Y_train=Y.loc[Ascendum.date<pd.Timestamp('06-22-2019')]
Y_val=Y.loc[Ascendum.date>=pd.Timestamp('06-22-2019')]

In [ ]:
### fitt the XGboost model 

model_xgboost=xgb.XGBRegressor()
model_xgboost.fit(X_train,Y_train)
xgb_prediction=model_xgboost.predict(X_val)      ## check the prediction

[12:54:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
## checking the errors with outliers

print("Mean absolute error: ",mean_absolute_error(Y_val,xgb_prediction))
print("Mean squared error: ",mean_squared_error(Y_val,xgb_prediction))
print("R2: ", r2_score(Y_val,xgb_prediction))

Mean absolute error:  16.761965376627316
Mean squared error:  10371.898160549903
R2:  0.9713864745524204
